<a href="https://colab.research.google.com/github/TanmayKumar-EngStud/DeepLearning_Basic-programs/blob/main/LoadingDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Python code for loading and refining Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

DataDir = "/content/drive/MyDrive/newSET"
Category = ["women","men"]
print('done')

done


In [ ]:
trainingData = []
IMG_SIZE = 250
def create_Training_Data():
    for cat in Category:
        path = os.path.join(DataDir, cat)
        class_num= Category.index(cat)
        for img in os.listdir(path):
            try:
                img_arr   = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                trainingData.append([new_array, class_num])
            except Exception as e:
                pass
create_Training_Data()            
print(len(trainingData))


3262


In [ ]:
import random 
random.shuffle(trainingData)

X= []
y= []
for features, label in trainingData:
    X.append(features)
    y.append(label)


X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)
print('done')

done


In [ ]:
import pickle 

pickle_out = open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

X = X/255.0
print('done')

done


## **Creating the model** 


```
# This the actual code of my Convolution network
```


---





In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, MaxPooling2D

## Once Again calling the X and y value to make it a numpy array

In [ ]:
X = np.array(X)
y =np.array(y)
print(X.shape[1:])

(250, 250, 1)


## Building the model
---
-

> Defining the function which creates model




In [ ]:
def createModel():
    model= Sequential()
    model.add(Conv2D(64,(3,3),input_shape= (250,250,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size= (2,2)))

    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size= (2,2)))

    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size= (2,2)))

    model.add(Flatten())
    model.add(Dense(64))


    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [ ]:
model = createModel()

model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics = ['accuracy'])
model.fit(X,y, batch_size = 16, validation_split=0.1, epochs = 5)

Epoch 1/5
184/184 [==============================] - 46s 88ms/step - loss: 0.8998 - accuracy: 0.5408 - val_loss: 0.6698 - val_accuracy: 0.6208
Epoch 2/5
184/184 [==============================] - 15s 83ms/step - loss: 0.6694 - accuracy: 0.5881 - val_loss: 0.6919 - val_accuracy: 0.5933
Epoch 3/5
184/184 [==============================] - 15s 83ms/step - loss: 0.6063 - accuracy: 0.6754 - val_loss: 0.6762 - val_accuracy: 0.6208
Epoch 4/5
184/184 [==============================] - 15s 83ms/step - loss: 0.5007 - accuracy: 0.7502 - val_loss: 0.7588 - val_accuracy: 0.6330
Epoch 5/5
184/184 [==============================] - 15s 83ms/step - loss: 0.3987 - accuracy: 0.8166 - val_loss: 0.8175 - val_accuracy: 0.6177


#**Saving the model**


---
by using the technique of checkpoint callback 



In [28]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dr = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only= True,
                                                 verbose =1)

model.fit(X,y, batch_size = 16, validation_split=0.1, epochs =1, callbacks=cp_callback)

184/184 [==============================] - 16s 85ms/step - loss: 0.3087 - accuracy: 0.8658 - val_loss: 1.1971 - val_accuracy: 0.6177

Epoch 00001: saving model to training_1/cp.ckpt


In [ ]:
!ls {checkpoint_dr}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


# Reloading the previous model
---
Using that model and testing

In [29]:
model = createModel()
model.load_weights('training_1/cp.ckpt')
print('done')


done
